In [68]:
import pandas as pd
from collections import Counter
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# Wire Transfers

In [69]:
frac = 1
wire = pd.read_csv('wire.csv', engine="pyarrow")
abm = pd.read_csv('abm.csv', engine="pyarrow")
cheque = pd.read_csv('cheque.csv', engine="pyarrow").sample(frac = frac)
eft = pd.read_csv('eft.csv', engine="pyarrow").sample(frac = frac)
emt = pd.read_csv('emt.csv', engine="pyarrow").sample(frac = frac)
card = pd.read_csv('card.csv', engine="pyarrow").sample(frac = frac)

In [70]:
kyc = pd.read_csv('kyc.csv', engine="pyarrow")
kyc_industry_codes = pd.read_csv('kyc_industry_codes.csv', engine="pyarrow")

In [71]:
print("# of Wire transactions: ", wire.shape[0])
print("# of ABM transactions: ", abm.shape[0])
print("# of Cheque transactions: ", cheque.shape[0])
print("# of EFT transactions: ", eft.shape[0])
print("# of EMT transactions: ", emt.shape[0])
print("# of Card transactions: ", card.shape[0])

# of Wire transactions:  4758
# of ABM transactions:  20657
# of Cheque transactions:  262943
# of EFT transactions:  488562
# of EMT transactions:  161221
# of Card transactions:  555956


In [72]:
dfs = {'wire': wire, 'abm': abm, 'cheque': cheque, 'eft': eft, 'emt': emt, 'card': card}

#Sorting the DFs by date and Time
for key in dfs.keys():
    if key == 'cheque':
        # dfs[key]['transaction_datetime'] = pd.to_datetime(dfs[key]['transaction_date'])
        dfs[key] = dfs[key].sort_values(by = 'transaction_date', ascending = True)
    else:
        # dfs[key]['transaction_datetime'] = pd.to_datetime(dfs[key]['transaction_date'] + ' ' + dfs[key]['transaction_time'] )
        dfs[key] = dfs[key].sort_values( by = ['transaction_date', 'transaction_time'] , ascending = [True, True])

cheque = dfs['cheque']
abm = dfs['abm']
card = dfs['card']
eft = dfs['eft']
emt = dfs['emt']
wire = dfs['wire']
dfs = {'wire': wire, 'abm': abm, 'cheque': cheque, 'eft': eft, 'emt': emt, 'card': card}

In [73]:
for dfs_key in dfs.keys():
    # print(dfs_key)
    # print(dfs[dfs_key].columns.values)
    if "merchant_category" in dfs[dfs_key].columns.values:
        print("merchant_category exists in ", dfs_key)
    # print('\n')

merchant_category exists in  card


In [74]:
#Collecting all unique customer IDs

wire_customers = dfs['wire']['customer_id'].unique()
abm_customers = dfs['abm']['customer_id'].unique()
cheque_customers = dfs['cheque']['customer_id'].unique()
eft_customers = dfs['eft']['customer_id'].unique()
emt_customers = dfs['emt']['customer_id'].unique()
card_customers = dfs['card']['customer_id'].unique()

all_customers= list(set(np.concatenate((wire_customers, abm_customers, cheque_customers, eft_customers, emt_customers, card_customers), axis=0)))


In [75]:
# Creating a DataFrame with all unique customer IDs to store the features
customer_stats = pd.DataFrame(all_customers, columns=['customer_id'])

In [76]:
# Collecting all the unique merchant IDs from the transactions and kyc data to create a merchant list
# merchant_list = np.array(kyc_industry_codes['industry_code'].values)
merchant_list = []
# merchant_list_2 = []
for df_key in dfs.keys():
    if 'merchant_category' in dfs[df_key].columns:
        dfs[df_key].loc[dfs[df_key]['merchant_category']=='other', 'merchant_category'] = 0
        dfs[df_key]['merchant_category'] = dfs[df_key]['merchant_category'].astype('int')
        merchant_list = np.unique(np.concatenate((merchant_list, dfs[df_key]['merchant_category'].values), axis=0))
        # merchant_list_2 = np.unique(np.concatenate((merchant_list_2, dfs[df_key]['merchant_category'].values), axis=0))
        

In [77]:
len(merchant_list)

58

In [78]:
merchant_list

array([   0., 4121., 4215., 4722., 4784., 4812., 4814., 4816., 4899.,
       4900., 5039., 5045., 5047., 5085., 5200., 5211., 5251., 5300.,
       5310., 5311., 5331., 5411., 5499., 5511., 5533., 5541., 5542.,
       5651., 5655., 5691., 5712., 5732., 5734., 5812., 5814., 5815.,
       5816., 5817., 5818., 5912., 5921., 5941., 5942., 5943., 5968.,
       5999., 6300., 7011., 7311., 7372., 7399., 7523., 7538., 7542.,
       8099., 8398., 8699., 9399.])

In [79]:
# Function to divide the date range into weekly and monthly ranges
def get_date_ranges(start_date, end_date):
    # Convert the start and end date to datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    start_date_week = start_date
    end_date_week = end_date
    
    start_date_month = start_date
    end_date_month = end_date
    
    # Adjust the start_date to the previous Monday if it's not already a Monday
    if start_date_week.weekday() != 0:
        start_date_week -= pd.Timedelta(days=start_date_week.weekday())
    
    # Adjust the end_date to the next Sunday if it's not already a Sunday
    if end_date_week.weekday() != 6:
        end_date_week += pd.Timedelta(days=(6 - end_date_week.weekday()))
    
    # Generate a range of dates from start_date to end_date with a frequency of 'W-MON' (weekly on Monday)
    week_starts = pd.date_range(start=start_date_week, end=end_date_week, freq='W-MON')
    
    # Create a list of tuples with start and end dates for each week
    weekly_ranges = []
    for start in week_starts:
        end = start + pd.Timedelta(days=6)
        weekly_ranges.append((start, end))
    
    # Adjust the start_date to the first day of the month
    start_date_month = start_date_month.replace(day=1)
    
    # Adjust the end_date to the last day of the month
    if end_date_month.days_in_month != end_date_month.day:
        end_date_month.replace(day=end_date_month.days_in_month)
        
    # end_date_month = (end_date_month + pd.offsets.MonthEnd(1)).normalize()
    
    # Generate a range of dates from start_date_month to end_date_month with a frequency of 'MS' (monthly start)
    month_starts = pd.date_range(start=start_date_month, end=end_date_month, freq='MS')
    
    # Create a list of tuples with start and end dates for each month
    monthly_ranges = []
    for start in month_starts:
        end = (start + pd.offsets.MonthEnd(1)).normalize()
        monthly_ranges.append((start, end))
    
    return weekly_ranges, monthly_ranges

In [80]:
# Create a dictionary to map industry codes to indices for merchnat category dissimilarity calculation
industry_codes_dict = {str(int(code)): i for i, code in enumerate(merchant_list)}

In [81]:
df_customers = {'wire': wire_customers, 'abm': abm_customers, 'cheque': cheque_customers, 'eft': eft_customers, 'emt': emt_customers, 'card': card_customers}

for customer in tqdm(all_customers):
    for df_key in dfs.keys():
        # Find the eariest and latest transaction date for the customer in this transaction type
        df = dfs[df_key]
       
        # if customer == 'SYNCID0000006876':
        #     print("SYNCID0000006876")
       
        # Setting the features to 0 if the customer is not present in the transactions       
        if customer not in df_customers[df_key]:
            customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_max_weekly_trx'] = 0
            customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_max_monthly_trx'] = 0
            customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_max_weekely_trx_avg_val'] = 0
            customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_max_monthly_trx_avg_val'] = 0
            customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_ecommerce_activity'] = 0
            customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_cash_activity'] = 0
            
            if 'merchant_category' in dfs[df_key].columns:
                customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_merchant_categories'] = 0
                customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_merchant_category_dissimilarity'] = 0
                customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_merchant_category_dissimilarity_std'] = 0
            
            continue
        
        max_weekly_trx = 0
        max_monthly_trx = 0
        max_weekely_trx_avg_val = 0
        max_monthly_trx_avg_val = 0
        
        customer_df = df[df['customer_id'] == customer]
        
        start_date = df['transaction_date'].min()
        end_date = df['transaction_date'].max()
        weekly_ranges, monthly_ranges = get_date_ranges(start_date, end_date)
        
        # Calculate the number of unique merchant categories for this customer
        if 'merchant_category' in dfs[df_key].columns:
                merchant_categories = Counter(customer_df['merchant_category'])
                customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_merchant_categories'] = len(merchant_categories.keys())
        # else:
        #     customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_merchant_categories'] = 0
        # Calculate the maximum number of transactions in a week and the average transaction value
        merchant_vecs = np.zeros((len(weekly_ranges), len(industry_codes_dict.keys())))
        
        for i, (start, end) in enumerate(weekly_ranges):
            period_df = customer_df[(customer_df['transaction_date'] >= pd.to_datetime(start).date()) & (customer_df['transaction_date'] <= pd.to_datetime(end).date())]
            if len(period_df) > max_weekly_trx:
                max_weekly_trx = len(period_df)
                max_weekely_trx_avg_val = period_df['amount_cad'].mean()
            
            # Calculate the number of transactions for each merchant category and make a vector for each week for dissimilarity calculation           
            merchant_category_flag = 0
            if 'merchant_category' in df.columns:    
                merchant_category_flag = 1
                weekly_merchant_categories = Counter(period_df['merchant_category'])
                for category in weekly_merchant_categories.keys():
                    merchant_vecs[i, industry_codes_dict[str(category)]] = weekly_merchant_categories[category]

        customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_max_weekly_trx'] = max_weekly_trx
        customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_max_weekely_trx_avg_val'] = max_weekely_trx_avg_val
    
        # Calculate the dissimilarity between merchant categories in consecutive months and store the mean and standard deviation
        non_zero_merchant_vecs = merchant_vecs[merchant_vecs.sum(axis=1) > 0]
        if merchant_category_flag:
            dissimilarity = np.zeros(non_zero_merchant_vecs.shape[0]-1)
            for i in range(1,non_zero_merchant_vecs.shape[0]):
                dissimilarity[i-1] = np.linalg.norm(non_zero_merchant_vecs[i] - non_zero_merchant_vecs[i-1])
            
            if len(dissimilarity) > 0:
                customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_merchant_category_dissimilarity'] = dissimilarity.mean()
                customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_merchant_category_dissimilarity_std'] = dissimilarity.std()
            else:
                customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_merchant_category_dissimilarity'] = 0
                customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_merchant_category_dissimilarity_std'] = 0

        # Doing the monthly calculations
        for i,(start, end) in enumerate(monthly_ranges):
            period_df = customer_df[(customer_df['transaction_date'] >= pd.to_datetime(start).date()) & (customer_df['transaction_date'] <= pd.to_datetime(end).date())]
            
            # Calculate the maximum number of transactions in a month and the average transaction value
            if len(period_df) > max_monthly_trx:
                max_monthly_trx = len(period_df)
                max_monthly_trx_avg_val = period_df['amount_cad'].mean()

        
        # Store the maximum number of transactions and average transaction value for the month       
        customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_max_monthly_trx'] = max_monthly_trx
        customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_max_monthly_trx_avg_val'] = max_monthly_trx_avg_val
        
        
        # Calculate the ecommerce activity of the customer  
        if 'ecommerce_ind' in df.columns:
            ecommerce_ind_dict = Counter(customer_df['ecommerce_ind'])
            customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_ecommerce_activity'] = ecommerce_ind_dict[True]/len(customer_df)
        else:
            customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_ecommerce_activity'] = 0

        #Calculate the percentage of transactions that cash transactions
        if 'cash_indicator' in df.columns:
            cash_ind_dict = Counter(customer_df['cash_indicator'])
            customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_cash_activity'] = cash_ind_dict[True]/len(customer_df)
        else:
            customer_stats.loc[customer_stats['customer_id'] == customer, df_key+'_cash_activity'] = 0

#save the customer stats to a csv file
customer_stats.to_csv('customer_stats.csv', index=False)

100%|██████████| 16226/16226 [33:12<00:00,  8.14it/s]


In [4]:
customer_stats = pd.read_csv('customer_stats.csv', engine="pyarrow").sample(frac = frac)
